In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import unicodedata

def normalizar_nome(texto):
    """Remove acentos, converte para maiúsculas e remove espaços extras"""
    if pd.isna(texto):
        return texto
    texto = str(texto)
    # Remove acentos
    texto = unicodedata.normalize('NFKD', texto)
    texto = texto.encode('ASCII', 'ignore').decode('ASCII')
    # Maiúsculas e remove espaços extras
    return texto.upper().strip()

# Carregando os arquivos
idade_tratado = pd.read_csv('./Dataset/idade_tratado.csv', sep=';', encoding='utf-8')
municipios_br = pd.read_csv('./Dataset/municipios_br.csv')

# Verificando as colunas
print("Colunas idade_tratado:", idade_tratado.columns.tolist())
print("Colunas municipios_br:", municipios_br.columns.tolist())
print("\nPrimeiras linhas municipios_br:")
print(municipios_br.head())

# Criando colunas normalizadas para melhor correspondência
idade_tratado['MUNICIPIO_NORM'] = idade_tratado['MUNICIPIO'].apply(normalizar_nome)
municipios_br['nome_NORM'] = municipios_br['nome'].apply(normalizar_nome)

# Verificando correspondências
print("\n--- VERIFICANDO CORRESPONDÊNCIAS ---")
print("Amostra idade_tratado:", idade_tratado['MUNICIPIO'].head().tolist())
print("Amostra municipios_br:", municipios_br['nome'].head().tolist())

# Criando geometria se houver coordenadas
if 'latitude' in municipios_br.columns and 'longitude' in municipios_br.columns:
    municipios_br['geometry'] = municipios_br.apply(
        lambda row: Point(row['longitude'], row['latitude']) 
        if pd.notna(row['longitude']) and pd.notna(row['latitude']) 
        else None, 
        axis=1
    )
    municipios_br = gpd.GeoDataFrame(municipios_br, geometry='geometry', crs='EPSG:4326')
    print("✓ Geometria criada com sucesso!")

# Fazendo o merge COM NORMALIZAÇÃO
dados_linkados = municipios_br.merge(
    idade_tratado,
    left_on='nome_NORM',
    right_on='MUNICIPIO_NORM',
    how='right'
)

# Convertendo para GeoDataFrame
dados_linkados = gpd.GeoDataFrame(dados_linkados, geometry='geometry', crs='EPSG:4326')

# Removendo linhas sem geometria
dados_com_geo = dados_linkados[dados_linkados['geometry'].notna()].copy()

print(f"\n--- RESULTADOS DO MERGE ---")
print(f"Total de registros idade_tratado: {len(idade_tratado)}")
print(f"Total após merge: {len(dados_linkados)}")
print(f"Registros com geometria: {len(dados_com_geo)}")
print(f"Taxa de sucesso: {len(dados_com_geo)/len(dados_linkados)*100:.1f}%")

if len(dados_com_geo) > 0:
    # Calculando o centroide
    dados_com_geo['centroide'] = dados_com_geo.geometry.centroid
    dados_com_geo['longitude_centroide'] = dados_com_geo['centroide'].x
    dados_com_geo['latitude_centroide'] = dados_com_geo['centroide'].y
    
    # Visualizando o resultado
    print("\n--- PRIMEIRAS LINHAS DO RESULTADO ---")
    print(dados_com_geo[['MUNICIPIO', 'nome', 'longitude_centroide', 'latitude_centroide']].head())
    
    # Salvando o resultado
    dados_com_geo.to_file('./Dataset/dados_linkados_com_centroide.gpkg', driver='GPKG')
    print("\n✓ Arquivo GPKG salvo com sucesso!")
    
    # Salvando também como CSV
    dados_csv = dados_com_geo.drop(columns=['geometry', 'centroide'])
    dados_csv.to_csv('./Dataset/dados_linkados_com_centroide.csv', index=False, sep=';', encoding='utf-8')
    print("✓ Arquivo CSV salvo com sucesso!")
else:
    print("\n❌ NENHUM registro foi linkado com sucesso!")

# Verificando municípios não encontrados
nao_encontrados = dados_linkados[dados_linkados['geometry'].isna()]
if len(nao_encontrados) > 0:
    print(f"\n⚠️ {len(nao_encontrados)} municípios NÃO foram encontrados:")
    print(nao_encontrados['MUNICIPIO'].unique()[:20])
    
    # Salvando lista de não encontrados
    nao_encontrados[['MUNICIPIO']].drop_duplicates().to_csv(
        './Dataset/municipios_nao_encontrados.csv', 
        index=False, 
        sep=';', 
        encoding='utf-8'
    )
    print("✓ Lista de não encontrados salva em: municipios_nao_encontrados.csv")

Colunas idade_tratado: ['MUNICIPIO', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos', '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos', '60 a 69 anos', '70 a 79 anos', '80 anos e mais', 'Total']
Colunas municipios_br: ['id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf', 'id_municipio_bcb', 'nome', 'capital_uf', 'id_comarca', 'id_regiao_saude', 'nome_regiao_saude', 'id_regiao_imediata', 'nome_regiao_imediata', 'id_regiao_intermediaria', 'nome_regiao_intermediaria', 'id_microrregiao', 'nome_microrregiao', 'id_mesorregiao', 'nome_mesorregiao', 'id_regiao_metropolitana', 'nome_regiao_metropolitana', 'ddd', 'id_uf', 'sigla_uf', 'nome_uf', 'nome_regiao', 'amazonia_legal', 'centroide']

Primeiras linhas municipios_br:
   id_municipio  id_municipio_6  id_municipio_tse  id_municipio_rf  \
0       5101837          510183               NaN              NaN   
1       1100809          110080             477.0            681.0   
2       1100338        

ValueError: Unknown column geometry